In [23]:
import numpy as np
from PIL import Image

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


In [24]:
transform =  transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    
])

In [4]:
train_data = torchvision.datasets.CIFAR10(root="./data",train=True,transform=transform,download=True)
test_data = torchvision.datasets.CIFAR10(root="./data",train=False,transform=transform,download=True)

train_loader = torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=True,num_workers=2)


100.0%


In [25]:
image,label = train_data[0]

In [26]:
image.size()

torch.Size([3, 32, 32])

In [27]:
class_names = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [28]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3,12,5) #[12,28,28]
        self.pool = nn.MaxPool2d(2,2) #[12,14,14] #make 2*2 pixels inti 1.
        self.conv2 = nn.Conv2d(12,24,5) #[24,10,10] -> [24,5,5] -> flatten
        self.fc1 = nn.Linear(24*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
        
        

        

In [29]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.01,momentum=0.9)

In [30]:
for epoch in range(30):
    print(f"Training epoch {epoch}...")

    running_loss = 0.0 

    for i,data in enumerate(train_loader):
        inputs,labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        
    print(f"loss : {running_loss/len(train_loader):.4f}")

Training epoch 0...
loss : 1.6333
Training epoch 1...
loss : 1.2617
Training epoch 2...
loss : 1.0915
Training epoch 3...
loss : 0.9838
Training epoch 4...
loss : 0.9048
Training epoch 5...
loss : 0.8375
Training epoch 6...
loss : 0.7937
Training epoch 7...
loss : 0.7492
Training epoch 8...
loss : 0.7142
Training epoch 9...
loss : 0.6819
Training epoch 10...
loss : 0.6494
Training epoch 11...
loss : 0.6355
Training epoch 12...
loss : 0.6124
Training epoch 13...
loss : 0.6016
Training epoch 14...
loss : 0.5841
Training epoch 15...
loss : 0.5700
Training epoch 16...
loss : 0.5593
Training epoch 17...
loss : 0.5570
Training epoch 18...
loss : 0.5502
Training epoch 19...
loss : 0.5259
Training epoch 20...
loss : 0.5301
Training epoch 21...
loss : 0.5284
Training epoch 22...
loss : 0.5212
Training epoch 23...
loss : 0.5155
Training epoch 24...
loss : 0.5095
Training epoch 25...
loss : 0.5132
Training epoch 26...
loss : 0.5136
Training epoch 27...
loss : 0.4960
Training epoch 28...
loss : 0.

In [31]:
torch.save(net.state_dict(),'trained_net.pth')

In [32]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [33]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images,labels =data
        outputs = net(images)
        _, predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100*correct/total

print(f"Accuracy : {accuracy}")
        

Accuracy : 63.05


In [38]:
new_transforms = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    
])

def load_image(img_path):
    image = Image.open(img_path)
    image = new_transforms(image)
    image = image.unsqueeze(0)
    return image

image_paths=['catTest1.jpg','deertest11.jpg','Planetest1.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output,1)
        print(f'Prediction {class_names[predicted.item()]}')

Prediction cat
Prediction deer
Prediction plane
